https://github.com/bentrevett/pytorch-sentiment-analysis?tab=readme-ov-file

https://colab.research.google.com/github/bentrevett/pytorch-sentiment-analysis/blob/main/1%20-%20Neural%20Bag%20of%20Words.ipynb#scrollTo=a5478fc3

https://colab.research.google.com/github/bentrevett/pytorch-sentiment-analysis/blob/main/2%20-%20Recurrent%20Neural%20Networks.ipynb

https://colab.research.google.com/github/bentrevett/pytorch-sentiment-analysis/blob/main/3%20-%20Convolutional%20Neural%20Networks.ipynb

https://colab.research.google.com/github/bentrevett/pytorch-sentiment-analysis/blob/main/4%20-%20Transformers.ipynb

In [1]:
#Collap Drive Bağlama
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# Uninstall existing versions
!pip uninstall torch torchtext -y

# Install compatible versions of torch and torchtext
!pip install torch==2.2.0 torchtext==0.17.0


Found existing installation: torch 2.4.0
Uninstalling torch-2.4.0:
  Successfully uninstalled torch-2.4.0
Found existing installation: torchtext 0.18.0
Uninstalling torchtext-0.18.0:
  Successfully uninstalled torchtext-0.18.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 109.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 5.9 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.0.0
    Uninstalling triton-3.0.0:
      Successfully uninstalled triton-3.0.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.20.5
    Uninstalling nvidia-nccl-cu12-2.20.5:
      

In [10]:
import torch
print(torch.__version__)

2.4.0+cu121


In [2]:
import torch
import torchtext
print(torch.__version__)
print(torchtext.__version__)

2.2.0+cu121
0.17.0+cpu


In [10]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import re
import nltk
from sklearn.metrics import accuracy_score

# Download necessary NLTK packages
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Define custom stop words based on your setup
stop_words = set(nltk.corpus.stopwords.words("turkish"))
stop_words.update(["bir", "film", "filmi", "filme", "filmde", "filmden", "filmin", "kadar", "bi", "ben"])  # Add your custom stop words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
# Define the tokenizer and text preprocessing function
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub("[^abcçdefgğhıijklmnoöprsştuüvyz ]", "", text)  # Remove non-Turkish alphabetic characters
    tokens = nltk.word_tokenize(text)  # Tokenize the text
    tokens = [word for word in tokens if word not in stop_words]  # Remove stop words
    lemma = nltk.WordNetLemmatizer()  # Initialize lemmatizer
    tokens = [lemma.lemmatize(word) for word in tokens]  # Lemmatize tokens
    return tokens

# Define a function to yield tokens for building the vocabulary
def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield preprocess_text(text)

In [11]:
# Define the path to the datasets
path = "/content/drive/MyDrive/Binovist  Sentiment Analysis/data/Hepsiburada Dataset 492k(%10 test)/"

# Load the train and test datasets
df_train = pd.read_csv(path + "train.csv")
df_test = pd.read_csv(path + "test.csv")

# Combine datasets into a dictionary for easier processing
dataframes = {'df_train': df_train, 'df_test': df_test}

# Define sets for affirmative and negative responses (if needed in interactive filtering processes)
yes_responses = {'evet', 'yes', 'y', '1', 'tamam', 'onay', 'olur', 'e'}
no_responses = {'hayır', 'no', 'n', '0', 'hayir', 'olmaz', 'h'}

In [12]:
# Map labels to numeric values
label_mapping = {'Positive': 2, 'Notr': 1, 'Negative': 0}

# Apply the label mapping to both datasets
for name, df in dataframes.items():
    # Ensure the label column exists and map it to numeric values
    if 'label' in df.columns:
        df['label_numeric'] = df['label'].map(label_mapping)
        print(f"{name} Hatasız Eklendi")
    else:
        print(f"Label column not found in {name}")

# Display the first few rows of the datasets to confirm changes
print("Train Dataset Sample:")
print(df_train.head())

print("\nTest Dataset Sample:")
print(df_test.head())

df_train Hatasız Eklendi
df_test Hatasız Eklendi
Train Dataset Sample:
                                                text     label  \
0  ürünü hepsiburadadan alalı 3 hafta oldu. orjin...  Positive   
1  ürünlerden çok memnunum, kesinlikle herkese ta...  Positive   
2      hızlı kargo, temiz alışveriş.teşekkür ederim.  Positive   
3               Çünkü aranan tapınak bu bölgededir .      Notr   
4  bu telefonu başlıca alma nedenlerim ise elimde...  Positive   

          dataset  label_numeric  
0  urun_yorumlari              2  
1  urun_yorumlari              2  
2  urun_yorumlari              2  
3            wiki              1  
4  urun_yorumlari              2  

Test Dataset Sample:
                                                text     label  \
0      Kral akbaba dikkat çekici renklere sahiptir .      Notr   
1   ısrarla korkutmayı başarıyor. sanki korku çok...  Positive   
2  Neşe ve Üzüntü köprünün kırılmaya başlamasıyla...      Notr   
3  i phone 5 ten sonra gene 4'' ekra

In [13]:
# Build the vocabulary using the training data
vocab = build_vocab_from_iterator(yield_tokens(df_train), specials=["<unk>", "<pad>"])
vocab.set_default_index(vocab["<unk>"])

ValueError: too many values to unpack (expected 2)

In [ ]:
# Define the model architecture
class SentimentAnalysisModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout, pad_idx):
        super(SentimentAnalysisModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers,
                            bidirectional=bidirectional, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        embedded = self.dropout(self.embedding(text))
        lstm_out, _ = self.lstm(embedded)
        hidden = self.dropout(torch.cat((lstm_out[:, -1, :self.lstm.hidden_size],
                                         lstm_out[:, 0, self.lstm.hidden_size:]), dim=1))
        return self.fc(hidden)

In [ ]:
# Model parameters
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 3  # 3 classes: Positive, Neutral, Negative
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

In [ ]:
# Initialize the model
model = SentimentAnalysisModel(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

# Load pre-trained embeddings
model.embedding.weight.data.copy_(TEXT.vocab.vectors)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
# Function to train the model
def train(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0
    epoch_acc = 0

    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = accuracy_score(batch.label.cpu(), predictions.argmax(1).cpu())
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc

    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [ ]:
# Function to evaluate the model
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0
    predictions_list = []
    labels_list = []

    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = accuracy_score(batch.label.cpu(), predictions.argmax(1).cpu())
            epoch_loss += loss.item()
            epoch_acc += acc
            predictions_list.extend(predictions.argmax(1).cpu().numpy())
            labels_list.extend(batch.label.cpu().numpy())

    print(classification_report(labels_list, predictions_list, target_names=['Negative', 'Neutral', 'Positive']))
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [ ]:
# Train the model
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, test_iterator, criterion)

    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

# Saving the trained model
torch.save(model.state_dict(), 'sentiment_model_pytorch.pth')
print("Model training completed and saved.")